In [2]:
import torch

In [3]:
def get_crystals_list(
        frac_coords, atom_types, lengths, angles, num_atoms):
    """
    args:
        frac_coords: (num_atoms, 3)
        atom_types: (num_atoms)
        lengths: (num_crystals)
        angles: (num_crystals)
        num_atoms: (num_crystals)
    """
    assert frac_coords.size(0) == atom_types.size(0) == num_atoms.sum()
    assert lengths.size(0) == angles.size(0) == num_atoms.size(0)

    start_idx = 0
    crystal_array_list = []
    for batch_idx, num_atom in enumerate(num_atoms.tolist()):
        cur_frac_coords = frac_coords.narrow(0, start_idx, num_atom)
        cur_atom_types = atom_types.narrow(0, start_idx, num_atom)
        cur_lengths = lengths[batch_idx]
        cur_angles = angles[batch_idx]

        crystal_array_list.append({
            'frac_coords': cur_frac_coords.detach().cpu().numpy(),
            'atom_types': cur_atom_types.detach().cpu().numpy(),
            'lengths': cur_lengths.detach().cpu().numpy(),
            'angles': cur_angles.detach().cpu().numpy(),
        })
        start_idx = start_idx + num_atom
    return crystal_array_list

In [4]:
data = torch.load("generated/DiffCSP_mp_20.pt", map_location='cpu')
crystals_list = get_crystals_list(
    data['frac_coords'], data['atom_types'], data['lengths'], data['angles'], data['num_atoms'])

In [5]:
from pymatgen.core.structure import Structure, Lattice
def get_structure(record):
    return Structure(
                lattice=Lattice.from_parameters(
                    *(record['lengths'].tolist() + record['angles'].tolist())),
                species=record['atom_types'], coords=record['frac_coords'], coords_are_cartesian=False)

In [6]:
from multiprocessing import Pool
with Pool() as pool:
    structures = pool.map(get_structure, crystals_list)

In [13]:
from functools import partial
all(map(lambda x: isinstance(x, Structure), structures))

True

In [17]:
from pathlib import Path
Path("dfdf.csv").with_suffix(".json.gz")

PosixPath('dfdf.json.gz')

In [6]:
import gzip
import pickle
from pathlib import Path
with open(Path("cache", "wychoffs_enumerated_by_ss.pkl.gz"), "rb") as f:
    wychoffs_enumerated_by_ss = pickle.load(f)[0]

In [7]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OMP_THREAD_LIMIT"] = "1"
from functools import partial
from data import structure_to_sites
with Pool() as pool:
    sites = pool.map(partial(structure_to_sites, wychoffs_enumerated_by_ss=wychoffs_enumerated_by_ss), structures)

spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib64/python3.12/site-packages/pymatgen/core/periodic_table.py:267: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


In [18]:
with gzip.open(Path("generated", "DiffCSP_mp_20.json.gz"), "wt") as f:
    import json
    json.dump(sites, f)

SyntaxError: expected ':' (1952972701.py, line 1)